## Exact Exchange Fraction
This notebook will check the dependence on the exact fraction of exact change on QM-CR values.

In [ ]:
from numpy import linspace
from os.path import join
geom = "renumber"
functionals = linspace(0.0, 1.0, 21)
bset = "pcseg-1"

Read in the system.

In [ ]:
from pyntchem.io import read_pdb
with open(join("input", geom + ".pdb")) as ifile:
    sys = read_pdb(ifile)

Basis set.

In [ ]:
from pyntchem.basis import BasisSet
basis = BasisSet(bset, sys.get_symlookup())

Guess matrix from basis set work.

In [ ]:
alp = join("work-basis", basis.name, basis.name + ".DensAlp.mtx")

In [ ]:
from pyntchem.calculator import JobscriptCalculator
calc = JobscriptCalculator(computer="Spring", skip=True, verbose=True)
args = {"nodes": 1, "tasks_per_node": 4, "omp": 9,  "queue": "winter2"}

Compute.

In [ ]:
from pyntchem.preprocessing import put_guess_matrix
from contextlib import suppress
from pyntchem.inputfile import Inputfile
from time import sleep

for fun in functionals:
    inp = Inputfile()
    # Setup the custom amount of exact exchange
    inp.set_custom_dft({"pbe": 1 - fun}, {"pbe": 1.0}, fun)

    inp.set_dft_prune_grid(50, 194)
    inp.set_scf_guess("readdens")
    inp["scf"].pulayperiod = 2
    inp["scf"].facdamp = 0.95
    inp["scf"].writeanal = True
    inp["scf"].vshift = 0

    inp["int2"].prelinkjthreshold = 1e-8
    inp["int2"].prelinkkthreshold = 1e-4
    
    with suppress(IOError):
        put_guess_matrix(join("work-frac"), 
                         str(fun), dens_alp_file=alp)
    
    calc.run(sys, inp, basis, name=str(fun), 
             run_dir=join("work-frac"),  **args)
    
while not calc.check_results(): sleep(10.0)
logfiles = {x: y.log for x, y in calc.calculations.items()}

Post Process.

In [ ]:
from pyntchem.postprocessing import NTChemTool
tool = NTChemTool()

purities = {}
for f in functionals:
    log = logfiles[str(f)]
    purities[f] = tool.run_compute_purity(sys, log)

In [ ]:
bond_orders = {}
for f in functionals:
    log = logfiles[str(f)]
    bond_orders[f] = tool.fragment_bond_order(sys, list(sys), list(sys), log)

Plot the results.

In [ ]:
from matplotlib import pyplot as plt
fig, axs = plt.subplots(1, 1, figsize=(5, 4))

order = ['WAT:1', 'WAT:2', 'WAT:3', 'WAT:4', 'WAT:5', 'WAT:6', 'MOL:7', 'MOL:8', 'MOL:9', 'MOL:10']
cmap = plt.get_cmap("coolwarm")

for f in functionals:
    if f == 1.0:
        axs.plot([purities[f][k] for k in order], color=cmap(f), label="Exact Exchange")
    elif f == 0:
        axs.plot([purities[f][k] for k in order], color=cmap(f), label="PBE Exchange")
    else:
        axs.plot([purities[f][k] for k in order], color=cmap(f))
    
axs.legend(loc="lower left", ncol=1)
axs.set_ylim(-0.06, 0)
axs.set_xticks(range(len(order)))
axs.set_xticklabels(order, rotation=90)
axs.tick_params(axis='both', which='major', labelsize=16)
axs.set_ylabel("Purity Indicator", fontsize=18)
fig.tight_layout()
fig.savefig("frac-pi.png", dpi=600)

Plot the results for bond order.

In [ ]:
relevant = [('WAT:1', 'MOL:7'), ('WAT:1', 'MOL:8'), 
            ('WAT:3', 'MOL:8'), ('WAT:5', 'WAT:3'), 
            ('WAT:5', 'WAT:4'), ('MOL:7', 'MOL:8'), 
            ('MOL:9', 'WAT:3'), ('MOL:9', 'MOL:8'), 
            ('MOL:10', 'MOL:9')]

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(5, 5))

for f in functionals:
    if f == 1.0:
        axs.plot([bond_orders[f][k[0]][k[1]] for k in relevant],
                  color=cmap(f), label="Exact Exchange")
    elif f == 0.0:
        axs.plot([bond_orders[f][k[0]][k[1]] for k in relevant],
                  color=cmap(f), label="PBE Exchange")
    else:
        axs.plot([bond_orders[f][k[0]][k[1]] for k in relevant],
                  color=cmap(f))

axs.legend(loc="upper left", ncol=2)
axs.set_xticks(range(len(relevant)))
axs.set_xticklabels(["-".join(x) for x in relevant], rotation=90)
axs.set_ylim(5e-3, 5)
axs.set_yscale("log")

axs.tick_params(axis='both', which='major', labelsize=16)
axs.set_ylabel("Fragment Bond Order", fontsize=18)
fig.tight_layout()
fig.savefig("frac-fbo.png", dpi=600)